<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [53]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/bankfut/final_BANKNIFTY_F2.csv")
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') # Converted to reqd format
data.sort_values(by='Date', inplace=True)
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time
start_time = pd.to_datetime('09:15:00', format='%H:%M:%S').time()
end_time = pd.to_datetime('15:30:00', format='%H:%M:%S').time()
filtered_data = data[(data['Time'] >= start_time) & (data['Time'] <= end_time)] #Filtered out the data that doesnt occur in the time range
filtered_data

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
813167,BANKNIFTY_F2,01-01-2018,15:20:00,25515.20,25515.20,25515.20,25515.20,80.0,NaN
809216,BANKNIFTY_F2,01-01-2018,12:38:00,25637.50,25637.50,25637.50,25637.50,40.0,NaN
809217,BANKNIFTY_F2,01-01-2018,12:39:00,25640.00,25640.00,25640.00,25640.00,40.0,NaN
809218,BANKNIFTY_F2,01-01-2018,13:20:00,25635.80,25635.80,25625.10,25625.10,120.0,NaN
809219,BANKNIFTY_F2,01-01-2018,13:24:00,25626.10,25626.10,25625.10,25625.10,120.0,NaN
...,...,...,...,...,...,...,...,...,...
562680,BANKNIFTY_F2,31-12-2021,10:42:00,35766.75,35769.00,35766.75,35769.00,325.0,109775.0
562681,BANKNIFTY_F2,31-12-2021,10:44:00,35775.00,35775.00,35765.00,35765.00,75.0,109775.0
562682,BANKNIFTY_F2,31-12-2021,10:45:00,35760.00,35772.55,35760.00,35772.55,75.0,109775.0
562683,BANKNIFTY_F2,31-12-2021,10:48:00,35766.80,35766.80,35766.80,35766.80,100.0,109750.0


In [54]:
import pandas as pd
df = pd.DataFrame(filtered_data)
df['Open Interest'] = df['Open Interest'].fillna(0) #Replaced Nan by 0
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'].astype(str), format='%d-%m-%Y %H:%M:%S', errors='coerce') #Concatenated Date and Time to set a unique index
df = df.set_index('DateTime')
df = df.dropna(subset=['Date', 'Time'])
all_days = df.index.normalize().unique()
full_range = pd.concat([pd.DataFrame(index=pd.date_range(start=f"{day} 09:16", end=f"{day} 15:30", freq='1T')) for day in all_days])
full_range = full_range.merge(df, how='left', left_index=True, right_index=True)
full_range.fillna(method='ffill', inplace=True)
full_range.reset_index(inplace=True)
full_range.rename(columns={'index': 'DateTime'}, inplace=True)
full_range['Date'] = full_range['DateTime'].dt.strftime('%d-%m-%Y')
full_range['Time'] = full_range['DateTime'].dt.strftime('%H:%M:%S')
full_range = full_range[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
full_range['DateTime'] = pd.to_datetime(full_range['Date'] + ' ' + full_range['Time'], format='%d-%m-%Y %H:%M:%S')
full_range.set_index('DateTime', inplace=True)

aggregation = {
    'Symbol': 'first',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Open Interest': 'last'
}

resampled_df = full_range.resample('5T').apply(aggregation)

# Reset index to split DateTime into Date and Time
resampled_df.reset_index(inplace=True)
resampled_df['Date'] = resampled_df['DateTime'].dt.strftime('%d-%m-%Y')
resampled_df['Time'] = resampled_df['DateTime'].dt.strftime('%H:%M:%S')

resampled_df.drop(columns=['DateTime'], inplace=True)
resampled_df = resampled_df[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]

resampled_df


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0
...,...,...,...,...,...,...,...,...,...
542663,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0
542664,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0
542665,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0
542666,BANKNIFTY_F2,28-02-2023,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0


In [55]:
resampled_df.head(60)

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.0,25643.1,25608.0,25643.1,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.5,25675.7,25631.5,25665.3,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.3,25665.3,25655.1,25655.1,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.7,25647.7,25647.7,25647.7,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.7,25647.7,25647.0,25647.0,440.0,0.0
5,BANKNIFTY_F2,01-01-2018,09:40:00,25655.1,25661.9,25655.0,25655.0,320.0,0.0
6,BANKNIFTY_F2,01-01-2018,09:45:00,25653.0,25653.0,25645.0,25650.0,440.0,0.0
7,BANKNIFTY_F2,01-01-2018,09:50:00,25645.0,25650.0,25645.0,25650.0,400.0,0.0
8,BANKNIFTY_F2,01-01-2018,09:55:00,25645.0,25650.0,25645.0,25650.0,400.0,0.0
9,BANKNIFTY_F2,01-01-2018,10:00:00,25642.0,25642.0,25642.0,25642.0,1680.0,0.0


In [56]:
df = pd.DataFrame(resampled_df)
df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0
...,...,...,...,...,...,...,...,...,...
542663,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0
542664,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0
542665,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0
542666,BANKNIFTY_F2,28-02-2023,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0


In [57]:

df['DateTime'] = pd.to_datetime(df['Date'].str.strip() + ' ' + df['Time'].str.strip(), format='%d-%m-%Y %H:%M:%S', errors='coerce')
df = df.dropna(subset=['DateTime'])  # Remove rows where DateTime could not be parsed
df.set_index('DateTime', inplace=True)
# Calculate typical price for VWAP
df.loc[:, 'Price'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate cumulative values for VWAP
cumulative_price_volume = (df['Price'] * df['Volume']).cumsum()
cumulative_volume = df['Volume'].cumsum()

df.loc[:, 'VWAP'] = cumulative_price_volume / cumulative_volume #Calculating for 1 min

# Aggregate the data into 5-minute intervals
resampled = df.resample('5T').agg({
    'Symbol': 'first',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Open Interest': 'last',
    'Price': 'last'
})
# Calculate cumulative values for the resampled data
resampled['Price'] = (resampled['High'] + resampled['Low'] + resampled['Close']) / 3
resampled_cumulative_price_volume = (resampled['Price'] * resampled['Volume']).cumsum()
resampled_cumulative_volume = resampled['Volume'].cumsum()

# Calculating VWAP for the 5-minute intervals
resampled['VWAP'] = resampled_cumulative_price_volume / resampled_cumulative_volume

resampled.reset_index(inplace=True)
resampled['Date'] = resampled['DateTime'].dt.strftime('%d-%m-%Y')
resampled['Time'] = resampled['DateTime'].dt.strftime('%H:%M')

resampled.drop(columns=['DateTime', 'Price'], inplace=True) # Dropped DateTime column and unnecessary columns
resampled = resampled[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest', 'VWAP']]

resampled.dropna(inplace=True)
resampled.tail(60)


<ipython-input-57-5f8b17ba838f>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resampled.dropna(inplace=True)


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP
542608,BANKNIFTY_F2,28-02-2023,10:35,40800.00,40808.00,40771.00,40771.00,1350.0,1022825.0,31835.727338
542609,BANKNIFTY_F2,28-02-2023,10:40,40757.20,40760.00,40740.00,40749.90,1250.0,1022725.0,31835.737262
542610,BANKNIFTY_F2,28-02-2023,10:45,40743.00,40743.00,40675.05,40691.05,4250.0,1022100.0,31835.770828
542611,BANKNIFTY_F2,28-02-2023,10:50,40702.30,40715.00,40698.95,40715.00,925.0,1022050.0,31835.778139
542612,BANKNIFTY_F2,28-02-2023,10:55,40713.60,40734.25,40703.55,40726.15,1275.0,1022125.0,31835.788229
542613,BANKNIFTY_F2,28-02-2023,11:00,40720.65,40740.00,40720.20,40738.10,950.0,1022075.0,31835.795757
542614,BANKNIFTY_F2,28-02-2023,11:05,40740.00,40750.00,40724.05,40750.00,1850.0,1022050.0,31835.810431
542615,BANKNIFTY_F2,28-02-2023,11:10,40729.95,40750.00,40724.90,40736.10,700.0,1022375.0,31835.815981
542616,BANKNIFTY_F2,28-02-2023,11:15,40728.10,40752.85,40720.00,40721.00,850.0,1022375.0,31835.822715
542617,BANKNIFTY_F2,28-02-2023,11:20,40721.00,40721.00,40710.00,40710.00,800.0,1022300.0,31835.829041


# MACD

In [58]:
resampled_df_copy = df.copy()

# Converted 'Date' and 'Time' columns to strings so that we can concatenate
resampled_df_copy['Date'] = resampled_df_copy['Date'].astype(str)
resampled_df_copy['Time'] = resampled_df_copy['Time'].astype(str)
resampled_df_copy['DateTime'] = pd.to_datetime(resampled_df_copy['Date'] + ' ' + resampled_df_copy['Time'], format='%d-%m-%Y %H:%M:%S')

resampled_df_copy = resampled_df_copy.dropna(subset=['DateTime'])
resampled_df_copy.set_index('DateTime', inplace=True) #Setting DateTime column as index

resampled_df_copy['EMA_12'] = resampled_df_copy['Close'].ewm(span=12, adjust=False).mean() #12-period EMA
resampled_df_copy['EMA_26'] = resampled_df_copy['Close'].ewm(span=26, adjust=False).mean() #26-period EMA

resampled_df_copy['MACD'] = resampled_df_copy['EMA_12'] - resampled_df_copy['EMA_26'] # MACD= 12_EMA - 26_EMA

resampled_df_copy.drop(['Date'	,'Time','EMA_12', 'EMA_26'], axis=1, inplace=True)

resampled_df_copy['Date'] = resampled_df_copy.index.strftime('%d-%m-%Y')
resampled_df_copy['Time'] = resampled_df_copy.index.strftime('%H:%M:%S')
resampled_df_copy = resampled_df_copy[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest', 'MACD']]
resampled_df_copy

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,MACD
DateTime,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,0.000000
2018-01-01 09:20:00,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,1.770940
2018-01-01 09:25:00,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,2.324572
2018-01-01 09:30:00,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,2.141525
2018-01-01 09:35:00,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,1.917867
...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,-3.619661
2023-02-28 15:15:00,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,-1.512115
2023-02-28 15:20:00,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,1.432681


# POC

In [60]:
data = resampled_df_copy.copy()
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d-%m-%Y %H:%M:%S')
data.reset_index(drop=True, inplace=True)
data.head()

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,MACD,DateTime
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.0,25643.1,25608.0,25643.1,480.0,0.0,0.000000,2018-01-01 09:15:00
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.5,25675.7,25631.5,25665.3,600.0,0.0,1.770940,2018-01-01 09:20:00
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.3,25665.3,25655.1,25655.1,240.0,0.0,2.324572,2018-01-01 09:25:00
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.7,25647.7,25647.7,25647.7,240.0,0.0,2.141525,2018-01-01 09:30:00
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.7,25647.7,25647.0,25647.0,440.0,0.0,1.917867,2018-01-01 09:35:00


In [61]:
df = pd.DataFrame(data)

df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d-%m-%Y')
df['Date'] = df['DateTime'].dt.date                     #Extracting the Date Part from 'DateTime'
aggregated = df.groupby(['Date', 'Close']).agg({'Volume': 'sum'}).reset_index()

poc = aggregated.loc[aggregated.groupby('Date')['Volume'].idxmax()] # POC for each date
poc.rename(columns={'Close': 'Point of Control'}, inplace=True)
poc

,Date,Point of Control,Volume
34,2018-01-01,25659.90,5440.0
51,2018-01-02,25417.90,14880.0
124,2018-01-03,25511.00,5560.0
168,2018-01-04,25500.00,28240.0
264,2018-01-05,25685.20,8880.0
...,...,...,...
91490,2023-02-22,40708.10,120300.0
91528,2023-02-23,40232.20,226450.0
91587,2023-02-24,40435.25,204700.0
91673,2023-02-27,40583.65,136900.0


In [62]:
df = pd.DataFrame(data)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['DateTime'].dt.date # Extracting Date from DateTime

daily_high_low = df.groupby('Date').agg({'High': 'max', 'Low': 'min'}).reset_index()
daily_high_low.rename(columns={'High': 'Local Maximum','Low':'Local Minimum'}, inplace=True)
daily_high_low

,Date,Local Maximum,Local Minimum
0,2018-01-01,25675.70,25481.30
1,2018-01-02,25554.60,25360.00
2,2018-01-03,25550.00,25432.80
3,2018-01-04,25573.30,25436.30
4,2018-01-05,25720.00,25575.00
...,...,...,...
1880,2023-02-24,40841.55,40308.05
1881,2023-02-25,NaN,NaN
1882,2023-02-26,NaN,NaN
1883,2023-02-27,40815.00,40228.05


####should i plot it??

In [63]:
df = resampled_df.copy()
df = pd.DataFrame(resampled_df)

df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')
df.set_index('DateTime', inplace=True)

def calculate_vah_val(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume #Here i have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2   # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']
    return pd.Series({'VAH': VAH, 'VAL': VAL})

result = df.groupby(df.index.date).apply(calculate_vah_val)
result

,VAH,VAL
2018-01-01,25660.200,25545.30
2018-01-02,25529.000,25422.45
2018-01-03,25544.500,25488.90
2018-01-04,25565.700,25510.00
2018-01-05,25711.500,25624.95
...,...,...
2023-02-24,40783.300,40425.00
2023-02-25,NaN,NaN
2023-02-26,NaN,NaN
2023-02-27,40805.650,40558.50
